In [1]:
import pandas as pd

peripheral_metadata = pd.read_parquet("peripheral_metadata.parquet")
peripheral_metadata

,id,title,publication_year,publication_date,language,type,is_retracted,cited_by_count,counts_by_year,countries_distinct_count,institutions_distinct_count,domain,primary_topic,fwci,is_in_top_1_percent,is_in_top_10_percent
0,https://openalex.org/W2099334955,American College of Rheumatology/European Leag...,2011,2011-03-01,en,article,False,907,"[{""year"": 2023, ""cited_by_count"": 22}, {""year""...",None,30,None,None,None,False,False
1,https://openalex.org/W2085860140,Myocardial perfusion and function single photo...,2006,2006-11-01,en,article,False,145,"[{""year"": 2022, ""cited_by_count"": 2}, {""year"":...",None,0,None,None,None,False,False
2,https://openalex.org/W1986666623,Phenotypic diversity in white adults with mode...,2013,2013-07-01,en,article,False,43,"[{""year"": 2023, ""cited_by_count"": 6}, {""year"":...",None,2,None,None,None,False,False
3,https://openalex.org/W2923521938,Bacteriological Analysis Of Lipsticks,2011,2011-01-20,en,article,False,0,[],1,1,None,None,None,False,False
4,https://openalex.org/W1561876765,RF QUADRUPOLE BEAM DYNAMICS DESIGN STUDIES,1979,1979-01-01,en,article,False,34,"[{""year"": 2013, ""cited_by_count"": 1}, {""year"":...",0,0,None,"{""id"": ""https://openalex.org/T11367"", ""display...",None,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12406469,https://openalex.org/W4408392782,Assessment of radiology resident competency in...,2025,2025-03-01,en,article,False,0,[],1,1,None,"{""id"": ""https://openalex.org/T11894"", ""display...",0.0,False,False
12406470,https://openalex.org/W4408392876,Hierarchical Deviant Processing in Auditory Co...,2025,2025-03-01,en,article,False,0,[],3,5,None,"{""id"": ""https://openalex.org/T10581"", ""display...",0.0,False,False
12406471,https://openalex.org/W4408392934,Discrete neighborhood difference pattern-based...,2025,2025-03-14,en,article,False,0,[],0,0,None,"{""id"": ""https://openalex.org/T10392"", ""display...",0.0,False,False
12406472,https://openalex.org/W586612870,Remedios Varo : the Mexican years,2012,2012-01-01,en,book,False,0,"[{""year"": 2025, ""cited_by_count"": 1}]",0,0,None,"{""id"": ""https://openalex.org/T13268"", ""display...",0.0,False,False


In [2]:
merged_df = pd.read_csv("[20260102]df_2024.csv")

In [3]:
core_edge_list = pd.read_parquet("core_edge_list.parquet")
core_edge_list

,source_id,target_id,is_core
0,https://openalex.org/W3015008549,https://openalex.org/W2797416907,False
1,https://openalex.org/W2797666765,https://openalex.org/W2136286439,False
2,https://openalex.org/W2797666765,https://openalex.org/W2407446548,False
3,https://openalex.org/W2260008062,https://openalex.org/W1803771685,False
4,https://openalex.org/W2240201974,https://openalex.org/W2161163003,False
...,...,...,...
69670374,https://openalex.org/W4408971015,https://openalex.org/W4407983367,False
69670375,https://openalex.org/W4408978700,https://openalex.org/W4386887384,False
69670376,https://openalex.org/W941782748,https://openalex.org/W2185546100,False
69670377,https://openalex.org/W941782748,https://openalex.org/W2962579100,False


In [4]:
import pandas as pd

# Group edge list by target_id and collect all source_ids citing it
cited_by_map = (
    core_edge_list.groupby("target_id")["source_id"]
    .apply(list)
    .to_dict()
)

# Map this back to merged_df based on 'id'
merged_df["cited_by"] = merged_df["id"].map(cited_by_map)

# Replace NaN with empty list if no citations
merged_df["cited_by"] = merged_df["cited_by"].apply(lambda x: x if isinstance(x, list) else [])


In [5]:
del core_edge_list

In [6]:
import dask.dataframe as dd
raw = dd.read_parquet("papers.parquet")

In [7]:


# Select only the id and publication_date columns from raw
pub_dates = raw[["id", "publication_date"]]

# Merge with merged_df on 'id'
merged_df = dd.merge(merged_df, pub_dates, on="id", how="left")

del raw, pub_dates

In [8]:
merged_df = merged_df.compute()

In [9]:
merged_df

,citation_year,id,in_degree,out_degree,pagerank,betweenness,component_id,component_n_nodes,total_nodes,total_edges,...,is_aging,topic_cleaned,Rank,SJR,SJR Best Quartile,H index,sjr_year,is_Q1,cited_by,publication_date
0,2024,https://openalex.org/W102223339,8.0,24.0,3.045789e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,1.0,dermatology,21040.0,0.202,Q3,10.0,NaN,0,"['https://openalex.org/W4298127855', 'https://...",2021-12-02
1,2024,https://openalex.org/W1012569218,11.0,24.0,4.977488e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,1.0,health services research,14766.0,0.364,Q3,16.0,NaN,0,"['https://openalex.org/W3000076270', 'https://...",2015-01-01
2,2024,https://openalex.org/W1009689743,21.0,28.0,7.990457e-08,5.642431e-09,59881374.0,12467057.0,12497001.0,57487527.0,...,1.0,infectious diseases,6511.0,0.824,Q2,83.0,NaN,0,"['https://openalex.org/W2986582043', 'https://...",2015-01-01
3,2024,https://openalex.org/W1023751202,8.0,13.0,3.825184e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,1.0,surgery,18449.0,0.253,Q3,30.0,NaN,0,"['https://openalex.org/W2893402110', 'https://...",2015-06-01
4,2024,https://openalex.org/W1037638342,61.0,18.0,1.577632e-07,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,NaN,cardiovascular medicine,533.0,3.527,Q1,163.0,NaN,1,"['https://openalex.org/W2994445582', 'https://...",2015-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74245,2024,https://openalex.org/W932196285,0.0,3.0,7.510800e-09,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,1.0,oncology,21040.0,0.202,Q3,10.0,NaN,0,[],2021-12-23
74246,2024,https://openalex.org/W975176531,14.0,34.0,4.622204e-08,5.135886e-09,59881374.0,12467057.0,12497001.0,57487527.0,...,1.0,hepatology,17374.0,0.279,Q3,29.0,NaN,0,"['https://openalex.org/W2171472230', 'https://...",2015-01-01
74247,2024,https://openalex.org/W976614830,6.0,15.0,3.158954e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,1.0,immunology,16457.0,0.306,Q3,70.0,NaN,0,"['https://openalex.org/W2747507649', 'https://...",2015-06-01
74248,2024,https://openalex.org/W998197047,1.0,3.0,1.389500e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,NaN,obstetrics and gynecology,14924.0,0.358,Q2,56.0,NaN,0,['https://openalex.org/W3217145840'],2015-06-06


In [10]:
merged_df.columns

Index(['citation_year', 'id', 'in_degree', 'out_degree', 'pagerank',
       'betweenness', 'component_id', 'component_n_nodes', 'total_nodes',
       'total_edges', 'n_components', 'title', 'publication_year',
       'countries_distinct_count', 'institutions_distinct_count',
       'journal_issn', 'fwci', 'journal_title', 'prim_topic',
       'missing_abstract', 'is_aging', 'topic_cleaned', 'Rank', 'SJR',
       'SJR Best Quartile', 'H index', 'sjr_year', 'is_Q1', 'cited_by',
       'publication_date'],
      dtype='object')

In [11]:
import ast
def parse_cited_by(x):
    """Turn string like "['url1', 'url2']" into a real list."""
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return parsed
        except Exception:
            return []
    return []

merged_df["cited_by"] = merged_df["cited_by"].apply(parse_cited_by)

In [12]:
import pandas as pd
import numpy as np

# ---- Convert publication_date in merged_df ----
merged_df["publication_date"] = pd.to_datetime(
    merged_df["publication_date"],
    errors="coerce",   # invalid/unparseable become NaT
    format="mixed"     # row-by-row parsing of different formats
)

# ---- Convert publication_date in peripheral_metadata ----
peripheral_metadata["publication_date"] = pd.to_datetime(
    peripheral_metadata["publication_date"],
    errors="coerce",
    format="mixed"
)

# ---- Optional sanity check ----
print("merged_df pub_date dtype:", merged_df["publication_date"].dtype)
print("peripheral_metadata pub_date dtype:", peripheral_metadata["publication_date"].dtype)

print("Number of bad dates in merged_df:", merged_df["publication_date"].isna().sum())
print("Number of bad dates in peripheral_metadata:", peripheral_metadata["publication_date"].isna().sum())


def diff_in_months(d1, d2):
    """Exact month difference between two dates."""
    if pd.isna(d1) or pd.isna(d2):
        return np.nan
    return (d2.year - d1.year) * 12 + (d2.month - d1.month) + (d2.day - d1.day) / 30.0


# ---- Rename peripheral_metadata for join clarity ----
peripheral_md = peripheral_metadata.rename(
    columns={"id": "cited_by", "publication_date": "citing_pub_date"}
)

# ---- Explode cited_by lists and join citing publication dates ----
exploded = merged_df[["id", "publication_date", "cited_by"]].explode("cited_by")
joined = exploded.merge(
    peripheral_md[["cited_by", "citing_pub_date"]],
    on="cited_by", how="left"
).drop_duplicates()

# ---- Aggregate sorted citing dates ----
agg = (
    joined.groupby(["id", "publication_date"], as_index=False)
          .agg(citing_dates=("citing_pub_date", lambda s: sorted(s.dropna())))
)

def compute_metrics(row):
    pub = row["publication_date"]
    dates = row["citing_dates"]

    # Drop citing dates that occur before publication (invalid metadata cases)
    dates = [d for d in dates if d >= pub]

    if not dates:
        return pd.Series({
            "months_to_1st": np.nan,
            "months_to_2nd": np.nan,
            "months_to_5th": np.nan,
            "months_to_first_after60d": np.nan,
            "censored_1st": True,
            "censored_2nd": True,
            "censored_5th": True,
            "censored_after60d": True
        })

    # Compute month lags
    months_all = [diff_in_months(pub, d) for d in dates]
    cutoff = pub + pd.Timedelta(days=60)
    after60 = [d for d in dates if d >= cutoff]

    return pd.Series({
        "months_to_1st": months_all[0] if len(months_all) >= 1 else np.nan,
        "months_to_2nd": months_all[1] if len(months_all) >= 2 else np.nan,
        "months_to_5th": months_all[4] if len(months_all) >= 5 else np.nan,
        "months_to_first_after60d": diff_in_months(pub, after60[0]) if after60 else np.nan,
        "censored_1st": len(months_all) < 1,
        "censored_2nd": len(months_all) < 2,
        "censored_5th": len(months_all) < 5,
        "censored_after60d": not bool(after60)
    })

# ---- Apply per article ----
metrics = agg.apply(compute_metrics, axis=1)
lags_by_article = pd.concat([agg[["id"]], metrics], axis=1)

# ---- Merge back into merged_df ----
merged_df = merged_df.drop(columns=[col for col in merged_df.columns if col.startswith("censored")], errors="ignore")
merged_df = merged_df.merge(lags_by_article, on="id", how="left")

merged_df


merged_df pub_date dtype: datetime64[ns]
peripheral_metadata pub_date dtype: datetime64[ns]
Number of bad dates in merged_df: 0
Number of bad dates in peripheral_metadata: 10


,citation_year,id,in_degree,out_degree,pagerank,betweenness,component_id,component_n_nodes,total_nodes,total_edges,...,cited_by,publication_date,months_to_1st,months_to_2nd,months_to_5th,months_to_first_after60d,censored_1st,censored_2nd,censored_5th,censored_after60d
0,2024,https://openalex.org/W102223339,8.0,24.0,3.045789e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W4298127855, https://ope...",2021-12-02,0.966667,8.966667,NaN,8.966667,False,False,True,False
1,2024,https://openalex.org/W1012569218,11.0,24.0,4.977488e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W3000076270, https://ope...",2015-01-01,23.000000,48.000000,59.966667,23.000000,False,False,False,False
2,2024,https://openalex.org/W1009689743,21.0,28.0,7.990457e-08,5.642431e-09,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W2986582043, https://ope...",2015-01-01,20.833333,30.833333,58.500000,20.833333,False,False,False,False
3,2024,https://openalex.org/W1023751202,8.0,13.0,3.825184e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W2893402110, https://ope...",2015-06-01,8.300000,31.000000,64.266667,8.300000,False,False,False,False
4,2024,https://openalex.org/W1037638342,61.0,18.0,1.577632e-07,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W2994445582, https://ope...",2015-06-24,2.233333,2.233333,21.633333,2.233333,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266591,2024,https://openalex.org/W932196285,0.0,3.0,7.510800e-09,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,[],2021-12-23,NaN,NaN,NaN,NaN,True,True,True,True
1266592,2024,https://openalex.org/W975176531,14.0,34.0,4.622204e-08,5.135886e-09,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W2171472230, https://ope...",2015-01-01,32.900000,37.166667,64.433333,32.900000,False,False,False,False
1266593,2024,https://openalex.org/W976614830,6.0,15.0,3.158954e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W2747507649, https://ope...",2015-06-01,26.733333,30.466667,67.000000,26.733333,False,False,False,False
1266594,2024,https://openalex.org/W998197047,1.0,3.0,1.389500e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,[https://openalex.org/W3217145840],2015-06-06,76.666667,NaN,NaN,76.666667,False,True,True,False


In [13]:

# round month columns to integers
for col in ["months_to_1st", "months_to_2nd", "months_to_5th", "months_to_first_after60d"]:
    merged_df[col] = merged_df[col].round().astype("Int64")  # keep NA support

merged_df

,citation_year,id,in_degree,out_degree,pagerank,betweenness,component_id,component_n_nodes,total_nodes,total_edges,...,cited_by,publication_date,months_to_1st,months_to_2nd,months_to_5th,months_to_first_after60d,censored_1st,censored_2nd,censored_5th,censored_after60d
0,2024,https://openalex.org/W102223339,8.0,24.0,3.045789e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W4298127855, https://ope...",2021-12-02,1,9,<NA>,9,False,False,True,False
1,2024,https://openalex.org/W1012569218,11.0,24.0,4.977488e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W3000076270, https://ope...",2015-01-01,23,48,60,23,False,False,False,False
2,2024,https://openalex.org/W1009689743,21.0,28.0,7.990457e-08,5.642431e-09,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W2986582043, https://ope...",2015-01-01,21,31,58,21,False,False,False,False
3,2024,https://openalex.org/W1023751202,8.0,13.0,3.825184e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W2893402110, https://ope...",2015-06-01,8,31,64,8,False,False,False,False
4,2024,https://openalex.org/W1037638342,61.0,18.0,1.577632e-07,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W2994445582, https://ope...",2015-06-24,2,2,22,2,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266591,2024,https://openalex.org/W932196285,0.0,3.0,7.510800e-09,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,[],2021-12-23,<NA>,<NA>,<NA>,<NA>,True,True,True,True
1266592,2024,https://openalex.org/W975176531,14.0,34.0,4.622204e-08,5.135886e-09,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W2171472230, https://ope...",2015-01-01,33,37,64,33,False,False,False,False
1266593,2024,https://openalex.org/W976614830,6.0,15.0,3.158954e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,"[https://openalex.org/W2747507649, https://ope...",2015-06-01,27,30,67,27,False,False,False,False
1266594,2024,https://openalex.org/W998197047,1.0,3.0,1.389500e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,[https://openalex.org/W3217145840],2015-06-06,77,<NA>,<NA>,77,False,True,True,False


In [14]:
temp_df = pd.read_parquet('result_df_0614.parquet')
temp_df = temp_df[['id','journal_title']]
merged_df = merged_df.merge(temp_df, on='id', how='left')

In [15]:
merged_df = merged_df.drop_duplicates(subset=['id'])
merged_df

,citation_year,id,in_degree,out_degree,pagerank,betweenness,component_id,component_n_nodes,total_nodes,total_edges,...,publication_date,months_to_1st,months_to_2nd,months_to_5th,months_to_first_after60d,censored_1st,censored_2nd,censored_5th,censored_after60d,journal_title_y
0,2024,https://openalex.org/W102223339,8.0,24.0,3.045789e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,2021-12-02,1,9,<NA>,9,False,False,True,False,International Journal of High Dilution Research
5,2024,https://openalex.org/W1012569218,11.0,24.0,4.977488e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,2015-01-01,23,48,60,23,False,False,False,False,International Journal of Pediatrics and Adoles...
16,2024,https://openalex.org/W1009689743,21.0,28.0,7.990457e-08,5.642431e-09,59881374.0,12467057.0,12497001.0,57487527.0,...,2015-01-01,21,31,58,21,False,False,False,False,World Journal of Hepatology
27,2024,https://openalex.org/W1023751202,8.0,13.0,3.825184e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,2015-06-01,8,31,64,8,False,False,False,False,Acta Ortopedica Brasileira
38,2024,https://openalex.org/W1037638342,61.0,18.0,1.577632e-07,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,2015-06-24,2,2,22,2,False,False,False,False,JACC: Cardiovascular Interventions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6977224,2024,https://openalex.org/W932196285,0.0,3.0,7.510800e-09,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,2021-12-23,<NA>,<NA>,<NA>,<NA>,True,True,True,True,International Journal of High Dilution Research
6977229,2024,https://openalex.org/W975176531,14.0,34.0,4.622204e-08,5.135886e-09,59881374.0,12467057.0,12497001.0,57487527.0,...,2015-01-01,33,37,64,33,False,False,False,False,Polish Journal of Pathology
6977240,2024,https://openalex.org/W976614830,6.0,15.0,3.158954e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,2015-06-01,27,30,67,27,False,False,False,False,Natural Product Communications
6977251,2024,https://openalex.org/W998197047,1.0,3.0,1.389500e-08,0.000000e+00,59881374.0,12467057.0,12497001.0,57487527.0,...,2015-06-06,77,<NA>,<NA>,77,False,True,True,False,Encephale


In [16]:
# Export merged_df as CSV
merged_df.to_csv("[20260102]merged_df_survival.csv", index=False)